<a href="https://colab.research.google.com/github/conorato/INF8215/blob/master/TP2_INF8215.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP2 - Recherche Adversarielle 
### Membres de l'équipe
    - Claudia Onorato (1845448)
    - William Harvey (1851388)
    - Mathieu Bélanger (1850591)




## Directives de remise
Le travail sera réalisé avec la  même équipe que pour le TP1. Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **6 novembre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barème
Partie 1: 25 points

Partie 2: 35 points

Partie 3: 15 points

Partie 4: 15 points

Partie 5: 10 points

Pour un total de 100 points possibles sur 100 points.


## Recherche Adversarielle

La recherche adversarielle est une méthode d’intelligence artificielle qui est utilisée pour prédire les actions d’un adversaire dans un jeu vidéo. Il consiste à construire un arbre avec les actions possibles suivi des réactions de l’adversaire à ces actions. Pour choisir l’action optimale, il existe plusieurs méthodes. Une très commune est la recherche minimax. Elle évalue la valeur des actions et maximise la valeur de celle du joueur et minimise la valeur de celle de l’adversaire.

### Rush Hour

Vous avez vu le jeu de puzzle, Rush Hour, dans le TP précédent. Dans le cadre de ce TP, un adversaire a été ajouté pour transformer Rush Hour en jeu adversarielle. À chaque tour, l’adversaire ajoute une roche dans la grille. Cette roche disparaît après 1 tour et empêche les voitures de se déplacer. Son but est d’obliger le joueur à faire le plus de coups possible et de l’empêcher de bouger complètement. Le joueur peut déplacer une voiture, selon les règles vues dans le tp précédent, par tour. Cette version de Rush Hour est un jeu zero-sum déterministe.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

### But

Votre but dans ce TP est de développer un algorithme minimax pour permettre à votre AI de terminer le niveau de Rush Hour **avec un minimun de mouvement**. Une version complétée des classes Rush Hour et State est fournie pour vous aider à faire le TP. Ces classes vont être réutilisées pour représenter les états à évaluer. Vous pouvez les remplacer par vos propres implémentations en Python (vous pouvez aussi réécrire le TP dans le langage de votre choix).  

## 1. Implémentation de Rush Hour (25pts)
Les classes suivantes sont complètes pour représenter les différents états d’une partie de Rush Hour normale.
Il manque la représentation des coups de l’adversaire : vous auriez à implémenter cette partie-là dans les classes touchées.

L’état du stationnement au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos` : un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente la première case occupée par la voiture);

- `c` : l’index de la voiture déplacée pour atteindre l’état courant à partir de l’état précédent;

- `d` : la direction du déplacement fait à partir de l’état précédent pour atteindre l’état courant (+1 : vers la droite ou vers le bas, -1 : vers la gauche ou vers le haut);

- `prev` : l’état précédent

- `score` : le score associé à l'état

- `rock` : la position de la roche dans la grille (ligne,colonne)

In [0]:
import numpy as np
import math
import copy
from collections import deque

NB_SQUARES = 6

class State:

    """
    Contructeur d'un état initial
    """
    def __init__(self, pos):
        """
        pos donne la position de la voiture i dans sa ligne ou colonne (première case occupée par la voiture);
        """
        self.pos = np.array(pos)

        """
        c,d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.c = self.d = self.prev = None

        self.nb_moves = 0
        self.score = None

        # TODO
        """
        rock est un tuple représentant la position (ligne,colonne) de la roche
        """
        self.rock = None

    """
    Constructeur d'un état à partir du mouvement (c,d)
    """
    def move(self, c, d):
        s = State(self.pos)
        s.prev = self
        s.pos[c] += d
        s.c = c
        s.d = d
        s.nb_moves = self.nb_moves + 1
        # TODO
        s.rock = self.rock
        return s

    def put_rock(self, rock_pos):
        # TODO
        s = State(self.pos)
        s.rock = rock_pos
        s.c = self.c
        s.d = self.d
        s.nb_moves = self.nb_moves
        return s
    
           
    def score_state(self, rh):
      """
      The player wins when its score is zero. 
      This heuristic can under and over evaluate the real score (unlike A*).
      """
      self.score = 0
      rh.init_positions(self)

      line = rh.move_on[0] # always on the 3rd line

      for column in range(self.pos[0] + rh.length[0], NB_SQUARES): # All columns from exit
        # Only penalize if there's a car in the way
        if not rh.free_pos[line, column]:
          car_index = rh.find_car_index_at_pos(column, self)
          if rh.is_truck(car_index):
            # penalize if it's a truck, because it can only move down to get out of the way
            self.score += 4
          else:
            # penalize 2 per occupied space over if it's a car
            self.score += 2*sum([not rh.free_pos[i, column] for i in range(0, self.pos[car_index])])
          
          # penalize 2 per occupied space below the vehicle
          self.score += 2*sum([not rh.free_pos[i, column] for i in range(self.pos[car_index] + rh.length[car_index], NB_SQUARES)])

      # penalize 2 per square the red car has to move to get to the exit
      self.score += 1 * (NB_SQUARES - (self.pos[0] + rh.length[0]))

    def success(self):
        return self.pos[0] == 4
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de voitures")
        
        return np.array_equal(self.pos, other.pos)
    
    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37*h + self.pos[i]
        return int(h)
    
    def __lt__(self, other):
        return (self.score) < (other.score)


La représentation du problème est faite par la classe **Rushhour** qui contient les champs suivants :

- `nbcars` : le nombre de voitures dans la grille;
- `horiz` : un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on` : un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color` : un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos` : une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouvent dans le même index dans chacun des vecteurs.

La fonction *`print_pretty_grid()`* sert à imprimer la grille en utilisant la première lettre des couleurs pour vous aider à mieux visualiser le problème.

In [0]:
class Rushhour:

    def __init__(self, horiz, length, move_on, color=None):
        self.nbcars = len(horiz)
        self.horiz = horiz
        self.length = length
        self.move_on = move_on
        self.color = color

        self.free_pos = None

    def init_positions(self, state):
        self.free_pos = np.ones((6, 6), dtype=bool)
        for i in range(self.nbcars):
            if self.horiz[i]:
                self.free_pos[self.move_on[i], state.pos[i]:state.pos[i]+self.length[i]] = False
            else:
                self.free_pos[state.pos[i]:state.pos[i]+self.length[i], self.move_on[i]] = False
        # TODO
        if state.rock is not None:
            self.free_pos[state.rock[0], state.rock[1]] = False

    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(self.nbcars):
            if self.horiz[i]:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[self.move_on[i], state.pos[i]+self.length[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[self.move_on[i], state.pos[i] - 1]:
                    new_states.append(state.move(i, -1))
            else:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[state.pos[i]+self.length[i], self.move_on[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[state.pos[i] - 1, self.move_on[i]]:
                    new_states.append(state.move(i, -1))
        return new_states

    def possible_rock_moves(self, state):
        self.init_positions(state)
        new_states =[]
        # TODO
        free_pos_coordinates = zip(*np.where(self.free_pos))

        for line, col in free_pos_coordinates:
          is_same_line_or_col = state.rock is not None and (line == state.rock[0] or col == state.rock[1])
          if line != 2 and not is_same_line_or_col:
              new_states.append(state.put_rock((line,col)))
        return new_states

    def print_pretty_grid(self, state):
        self.init_positions(state)
        grid= np.chararray((6, 6))
        grid[:]='-'
        for car in range(self.nbcars):
            for pos in range(state.pos[car], state.pos[car] +self.length[car]):
                if self.horiz[car]:
                    grid[self.move_on[car]][pos] = self.color[car][0]
                else:
                    grid[pos][self.move_on[car]] = self.color[car][0]
        if state.rock is not None: # Ajout
            grid[state.rock] = 'x'
        print(grid)

    def find_car_index_at_pos(self, col, state):
      """
      Only supports finding car on the line 2 (where the red car is)
      """
      RED_CAR_LINE = 2
      if self.free_pos is None or self.free_pos[RED_CAR_LINE, col]:
        print("No car at that position")
        return None

      for line in range(RED_CAR_LINE+1):
        # Can only be one car at that position and moving on that column
        car_index = next((i for i, pos in enumerate(state.pos) if pos == line and self.move_on[i] == col and not self.horiz[i]), None)
        if car_index is not None and (line != 0 or self.is_truck(car_index)):
          return car_index
      
      return None

    def is_truck(self, car_index):
      return self.length[car_index] == 3


### 1.1 Implémentation

1. Modifier la classe State pour qu'elle contienne la position de la roche ainsi que la fonction *`put_rock()`* pour ajouter une nouvelle roche et enlever l'ancienne. Les roches sont représentées par un tuple (ligne,colonne). La fonction retourne un nouvel objet State.

2. Modifier la fonction *`init_position()`* de la classe Rushhour pour qu'elle tienne compte de l'emplacement des roches.

3. Implémenter la fonction *`possible_rock_moves()`* qui permet de trouver tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant dans la classe Rushhour. L'adversaire ne peut ni mettre de roche sur la ligne 2 ni mettre deux roches consécutives sur la même ligne ou colonne.

Utiliser la fonction *testRocks()* pour vérifier que vous avez bien pris en compte les roches. Vous devriez avoir ces résultats:

```
État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'x' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False False False False False]
 [False False False False False False]
 [False False False False  True False]]

```

Utiliser la fonction *testPossibleRockMoves()* pour vérifier que vous trouvez bien tous les coups possibles pour l'adversaire. Vous devriez trouver 7 et 3 mouvements possibles.

In [0]:
def testRocks():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s0 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    
    s1= s0.put_rock((4,4))
    s2 = s1.put_rock((3,2)) 
    
    print("État initial")
    rh.print_pretty_grid(s0)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 4-4")
    rh.print_pretty_grid(s1)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 3-2")
    rh.print_pretty_grid(s2)
    print(rh.free_pos)
    print('\n')

testRocks()

État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'

In [0]:
def testPossibleRockMoves():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    sols = rh.possible_rock_moves(s)
    print(len(sols))
    s1 = s.put_rock((3,4))
    sols = rh.possible_rock_moves(s1)
    print(len(sols))

testPossibleRockMoves()
  

7
3


## 2. Implémentation d'une recherche minimax (35pts)
L'algorithme minimax est l'implémentation la plus commune pour résoudre un jeu adversarielle avec des mouvements limités définis. Cet algorithme est basé sur le principe de construire un arbre avec les différents états possibles selon les actions de chaque joueur (AI et adversaire). L'algorithme veut maximiser les étages où le AI joue et minimiser les étages où l'adversaire joue. Dans ce cas-ci, l'AI déplace les voitures et l'adversaire place les roches. Dans le cadre du TP, l'algorithme n'explore pas jusqu'aux feuilles de l'arbre. Il s'arrête à une profondeur donnée.

La résolution du problème est faite par la classe **MiniMaxSearch** qui contient les champs suivants:

- `rushhour`: la classe Rushhour qui contient les informations sur la grille;
- `state`: la classe State avec l'état actuel de la grille;
- `search_depth`: la profondeur maximale d'exploration de l'algorithme;


In [0]:
from operator import attrgetter

class MiniMaxSearch:
    def __init__(self, rushHour, initial_state, search_depth):
        self.rushhour = rushHour
        self.state = initial_state
        self.search_depth = search_depth
        self._already_visited_states = set()
        self._visited_nodes = 0
        self._alpha = 0
        self._beta = 0

    def _remove_already_visited_states(self, possible_moves):
        return [possible_move for possible_move in possible_moves if possible_move not in self._already_visited_states]

    def minimax_1(self, current_depth, current_state): 
        #TODO
        if current_depth == self.search_depth:
          return current_state

        possible_moves = self._remove_already_visited_states(self.rushhour.possible_moves(current_state))
        best_moves = [self.minimax_1(current_depth+1, move) for move in possible_moves]

        for move in best_moves:
          if move.score is None:
            move.score_state(self.rushhour)
        minimum_score_state = min(best_moves, key=attrgetter('score'))

        if current_depth == 0:
          print("looking to score", minimum_score_state.score)
          possible_move_index = best_moves.index(minimum_score_state)
          return possible_moves[possible_move_index]
        else:
          return minimum_score_state

    def minimax_2(self, current_depth, current_state, is_max): 
        #TODO
        if current_depth == self.search_depth:
          return current_state

        if is_max:
          # Tour de la roche
          possible_moves = self.rushhour.possible_rock_moves(current_state)
          self._visited_nodes += len(possible_moves)
          best_moves = [self.minimax_2(current_depth+1, move, False) for move in possible_moves]
        else:
          # Tour du joueur
          possible_moves = self._remove_already_visited_states(self.rushhour.possible_moves(current_state))
          self._visited_nodes += len(possible_moves)
          best_moves = [self.minimax_2(current_depth+1, move, True) for move in possible_moves]

        for move in best_moves:
          if move.score is None:
            move.score_state(self.rushhour)

        minimum_score_state = max(best_moves, key=attrgetter('score')) if is_max else min(best_moves, key=attrgetter('score'))

        if current_depth == 0:
          print("looking to score", minimum_score_state.score)
          possible_move_index = best_moves.index(minimum_score_state)
          return possible_moves[possible_move_index]
        else:
          return minimum_score_state

    def minimax_pruning(self, current_depth, current_state, is_max):
        #TODO
        if current_depth == self.search_depth:
          return current_state

        if is_max:
          # Tour de la roche
          possible_moves = self.rushhour.possible_rock_moves(current_state)
          self._visited_nodes += len(possible_moves)
          best_moves = [self.minimax_pruning(current_depth + 1, move, False) for move in possible_moves]
        else:
          # Tour du joueur
          possible_moves = self._remove_already_visited_states(self.rushhour.possible_moves(current_state))
          self._visited_nodes += len(possible_moves)
          best_moves = [self.minimax_pruning(current_depth + 1, move, True) for move in possible_moves]

        for move in best_moves:
          if move.score is None:
            move.score_state(self.rushhour)

        score_state = None
        if is_max:
          for move in best_moves:
            score_state = max(score_state, move, key=attrgetter('score')) if score_state else move
            if score_state.score >= self._beta:
              break
            self._alpha = max(self._alpha, score_state.score)
        else:
          for move in best_moves:
            score_state = min(score_state, move, key=attrgetter('score')) if score_state else move
            if score_state.score <= self._alpha:
              break
            self._beta = min(self._beta, score_state.score)

        if current_depth == 0:
          print("looking to score", score_state.score)
          possible_move_index = best_moves.index(score_state)
          return possible_moves[possible_move_index]
        else:
          return score_state

    def expectimax(self, current_depth, current_state, is_max):
        #TODO
        return best_move

    def decide_best_move_1(self, current_state):
        #TODO
        return self.minimax_1(0, current_state)

    def decide_best_move_2(self, current_state, is_max):
        #TODO
        return self.minimax_2(0, current_state, is_max)

    def decide_best_move_pruning(self,  current_state, is_max):
        # TODO
        return self.minimax_pruning(0, current_state, is_max)

    def decide_best_move_expectimax(self, is_max):
        # TODO
        pass

    def solve(self, state, is_singleplayer, pruning = False):
        #TODO
        if is_singleplayer:
          while not state.success():
            next_state = self.decide_best_move_1(state)
            self.print_move(False, next_state)
            self.rushhour.print_pretty_grid(next_state)
            self._already_visited_states.add(next_state)
            state = next_state
        else:
          while not state.success():
            next_state = self.decide_best_move_pruning(state, False) if pruning else self.decide_best_move_2(state, False)
            self.print_move(False, next_state)
            self._already_visited_states.add(next_state)
            state = next_state

            next_state = self.decide_best_move_pruning(state, True) if pruning else self.decide_best_move_2(state, True)
            self.print_move(True, next_state)
            self.rushhour.print_pretty_grid(next_state)
            state = next_state
        print('%i visited nodes.' % self._visited_nodes)

    def print_move(self, is_max, state):
        #TODO
        if is_max:
          print("%i. Roche dans la case %i-%i." % (int(state.nb_moves), state.rock[0], state.rock[1]))
        else:
          self.print_move_player(state)

    def print_move_player(self, state):
        direction_labels = [["la droite", "la gauche"], ["le bas", "le haut"]]
        direction = 0 if state.d == 1 else 1 # Converts state.d from -1 to 1 (go up or go left) and from 1 to 0 (go down or go right)
        direction_label = direction_labels[0][direction] if self.rushhour.horiz[state.c] else direction_labels[1][direction]

        print("%i. Voiture %s vers %s" % (int(state.nb_moves), self.rushhour.color[state.c], direction_label))

### 2.1 Implémentation simple
1. Implémenter la fonction *`score_state()`* de la classe **State**. Elle affecte la valeur de l'état à son paramètre score. L'état n'est pas nécessairement final. Utiliser l'heuristique qui vous semble la plus pertinente.

2. Implémenter la fonction  *`minimax_1()`*. Cette fonction contient la logique de l'algorithme minimax pour un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant.

3. Implémenter la fonction *`decide_best_move_1()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à un joueur.

4. Implémenter la fonction *`solve()`*. Cette fonction résout un problème de Rush Hour avec le nombre minimal de coups.

4. Implémenter la fonction *`print_move()`* pour imprimer le coup fait. Ex. "Voiture orange vers le haut" ou "Roche dans la case 2-1".

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

Utiliser la fonction _test_print_move()_ pour vérifier votre implémentation de _print_move()_. 

In [0]:
def test_print_move():
    rh = Rushhour([True], [2], [2], ["rouge"])
    s = State([0])
    s = s.put_rock((3,1)) # Roche dans la case 3-1
    s = s.move(0, 1) # Voiture rouge vers la droite

    algo = MiniMaxSearch(rh, s, 1)
    algo.print_move(True, s)
    algo.print_move(False, s)

test_print_move()

1. Roche dans la case 3-1.
1. Voiture rouge vers la droite


Pour vous aider dans l'implémentation de votre heuristique, 3 exemples de problèmes de Rush Hour sont fournis.

In [0]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,2) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.rushhour.print_pretty_grid(s)
algo.solve(s, True)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
looking to score 8
1. Voiture rouge vers la droite
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'r' b'r' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
looking to score 6
2. Voiture orange vers le haut
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'o' b'r' b'r' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'j' b'j' b'j' b'-']]
looking to score 6
3. Voiture vert vers le bas
[[b'-' b'-' b'-' b'-

In [0]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 4)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
looking to score 11
1. Voiture lime vers la gauche
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'l' b'l' b'-']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
looking to score 11
2. Voiture vert vers la droite
[[b'-' b'v' b'v' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'l' b'l' b'-']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
looking to score 11
3. Voiture vert vers la droite
[[b'-' b'-' b'v' b'v' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'l' b'l' b'-']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
looking to scor

In [0]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,1)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
looking to score 19
1. Voiture rouge vers la droite
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'-' b'r' b'r' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
looking to score 19
2. Voiture 1 vers le bas
[[b'-' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'1' b'r' b'r' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
looking to score 19
3. Voiture 2 vers la gauche
[[b'-' b'-' b'2' b'2' b'2' b'-']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'1' b'r' b'r' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
looking to score 17
4. 

### 2.2 Implémentation adversarielle

1. Implémenter la fonction  *`minimax_2()`*. Cette fonction contient la logique de l'algorithme minimax pour deux joueurs et retourne le meilleur coup à prendre à partir de l'état courant.

2. Implémenter la fonction *`decide_best_move_2()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.

3. Modifier la fonction *`solve()`* pour qu'elle puisse résoudre des problèmes à deux joueurs.

4. Modifier la fonction *`score_state()`* de la classe **State** si elle ne tient pas déjà compte des roches.

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

 Le nombre donné de coups pour les 3 exemples suivants correspond à une partie sans adversaire. Le nombre de coups de plus dépend de l'implémentation de votre heuristique.

In [0]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,4)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
looking to score 10
1. Voiture rouge vers la droite
looking to score 8
1. Roche dans la case 0-0.
[[b'x' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'r' b'r' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
looking to score 10
2. Voiture vert vers le bas
looking to score 8
2. Roche dans la case 1-1.
[[b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'x' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'r' b'r' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
looking to score 8
3. Voiture vert vers le bas
looking to score 8
3. Roche dans la case 0-0.
[[b'x' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'-' b'b']
 [b'-' b'-' b'r'

In [0]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
looking to score 17
1. Voiture vert vers la droite
looking to score 17
1. Roche dans la case 0-0.
[[b'x' b'v' b'v' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
looking to score 17
2. Voiture vert vers la droite
looking to score 17
2. Roche dans la case 1-1.
[[b'-' b'-' b'v' b'v' b'-' b'j']
 [b'm' b'x' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
looking to score 17
3. Voiture vert vers la droite
looking to score 17
3. Roche dans la case 0-0.
[[b'x' b'-' b'-' b'v' b'v' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' 

In [0]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,4)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
looking to score 21
1. Voiture rouge vers la droite
looking to score 21
1. Roche dans la case 0-2.
[[b'1' b'-' b'x' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'-' b'r' b'r' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
looking to score 21
2. Voiture 1 vers le bas
looking to score 19
2. Roche dans la case 1-1.
[[b'-' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'x' b'-' b'3' b'-' b'5']
 [b'1' b'r' b'r' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
looking to score 19
3. Voiture 2 vers la gauche
looking to score 17
3. Roche dans la case 0-0.
[[b'x' b'-' b'2' b'2' b'2' b'-']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'1' b'r' b'r

## 3. Élagage Alpha-Beta (15pts)
Pour accélérer la prise de décision, un élagage est effectué sur l'arbre. Cela permet de ne pas parcourir tout l'arbre. L'élagage alpha-beta est l'implémentation la plus fréquente pour résoudre ce genre de problème. En passant des valeurs d’alpha et de bêta récursivement, l'algorithme connaît la meilleure valeur trouvée à date. Les branches jugées non intéressantes, c'est-à-dire qu'elles ne peuvent pas produire un meilleur résultat que celui déjà trouvé, sont coupées dans le processus.

 
### 3.1 Implémentation

1. Implémenter la fonction *`minimax_pruning()`* pour qu'elle n'explore pas des branches inutiles lors d'une partie à deux joueurs.
2. Implémenter la fonction *`decide_best_move_pruning()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Vérifier que cette nouvelle méthode visite bel et bien moins de noeuds. **Montrer cette information dans un tableau**.

In [0]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,4)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, pruning=True)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
[<__main__.State object at 0x7effcede8400>, <__main__.State object at 0x7effcede8c88>, <__main__.State object at 0x7effcede8b70>, <__main__.State object at 0x7effcede8748>, <__main__.State object at 0x7effcede8b38>, <__main__.State object at 0x7effcede8828>, <__main__.State object at 0x7effcede8518>, <__main__.State object at 0x7effcede8be0>, <__main__.State object at 0x7effcede85f8>, <__main__.State object at 0x7effcede89e8>, <__main__.State object at 0x7effcede88d0>, <__main__.State object at 0x7effcede8860>]
[<__main__.State object at 0x7effcede8748>, <__main__.State object at 0x7effcede8b38>, <__main__.State object at 0x7effcede8828>, <__main__.State object at 0x7effcede8518>, <__main__.State object at 0x7effcede8be0>, <__main__.State object at 0x7eff

In [0]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, pruning=True)
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
[<__main__.State object at 0x7effcee8ccc0>, <__main__.State object at 0x7effcef6bcc0>, <__main__.State object at 0x7effcef6b8d0>, <__main__.State object at 0x7effcef6bc88>, <__main__.State object at 0x7effcef6be10>, <__main__.State object at 0x7effcef6b080>, <__main__.State object at 0x7effcef6b470>, <__main__.State object at 0x7effcef6b5c0>, <__main__.State object at 0x7effcef6b128>, <__main__.State object at 0x7effcef6b668>, <__main__.State object at 0x7effcede8da0>, <__main__.State object at 0x7effcede8e48>, <__main__.State object at 0x7effcede88d0>, <__main__.State object at 0x7effcede8748>, <__main__.State object at 0x7effcede8518>]
[<__main__.State object at 0x7effcef6b7b8>, <__main__.State object at 0x7effcef6be48>, <__main__.State object at 0x7eff

In [0]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,4)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, pruning=True)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
[<__main__.State object at 0x7effcd583f28>, <__main__.State object at 0x7effcd583f98>, <__main__.State object at 0x7effcd583f60>, <__main__.State object at 0x7effcd583d68>, <__main__.State object at 0x7effcd583e10>, <__main__.State object at 0x7effcd583e80>]


TypeError: ignored

## 4. Expectimax (15 pts)
L'expectimax est une variante de la recherche minimax. Elle est utilisée lorsqu'il y a une composante de hasard lors de la prise de décision. Depuis le début du TP, il a été supposé que l'adversaire prenait toujours le coup optimal. Maintenant, l'adversaire va faire un coup aléatoirement. Pour résoudre ce cas-là, la recherche expectimax sera utilisée.

### 4.1 Implémentation 

1. Implémenter la fonction *`minimax_expectimax()`* pour qu'elle implémente l'algorithme de recherche. Considérer une probabilité égale pour tous les coups.
2. Implémenter la fonction *`decide_best_move_expectimax()`*. Cette fonction trouve et exécute le meilleur coup pour une partie avec un adversaire imprévisible.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Modifier les valeurs de probabilités associées aux coups pour donner une vision optimiste et une vision pessimiste à l'AI. Comparer la performance du AI entre les 3 visions (aléatoire, optimiste, pessimiste). Attention l'adversaire continue à prendre ses décisions au hasard. **Montrer ces informations dans un tableau et les analyser**. 

In [0]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

In [0]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

In [0]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

## 5. Comparaison (10pts)
Comparez la performance la recherche minimax avec la recherche expectimax lorsque l'adversaire est optimale et lorsque l'adversaire est aléatoire. Pour un adversaire aléatoire, prenez une vision neutre. Roulez plusieurs fois les algorithmes (vous pouvez aussi reprendre les tests du TP1) et notez si c'est une victoire ou une défaire pour l'AI ainsi que le nombre de coups moyen, minimal et maximal. Basez-vous sur le nombre de coups optimaux pour résoudre les problèmes pour déterminer un seuil de victoire et de défaite pour l'AI. Précisez votre seuil. **Mettez le tout dans un tableau et analysez vos résultats**. 